In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
import plotly.express as px

In [2]:
from PIL import Image
from stqdm import stqdm
class ClusterImage:
    def __init__(self, image_obj):
        self.image_obj = image_obj
        self.shape = np.shape(np.array(self.image_obj))
        self.image_array = np.reshape(np.array(self.image_obj),(self.shape[0] * self.shape[1], self.shape[2]) )
        self.clustered_image_array = np.reshape(np.array(self.image_obj),(self.shape[0] * self.shape[1], self.shape[2]) )
        self.cluster_image_obj = None

    def get_clustered_image(self, clusters = 10, clustering_type = "KMeans"):
        match clustering_type:
            case "KMeans":
                # st.write('kmeans')
                kmeans = KMeans(n_clusters=clusters, random_state=0, max_iter = 10, n_init=1).fit(self.image_array)
                for i, pixel in enumerate(tqdm(self.image_array)):
                    cluster = kmeans.labels_[i]
                    self.clustered_image_array[i] = [int(samp) for samp in kmeans.cluster_centers_[cluster] ]
        
        self.cluster_image_obj = Image.fromarray(np.reshape(self.clustered_image_array, self.shape))

    def graph_color_space(self, clustered_colors=False):
        if clustered_colors:   
            image_info = self.clustered_image_array
            TITLE = "Reduced Image Colorspace"
        else:
            image_info = self.image_array
            TITLE = "Original Image Colorspace"
        colors = np.unique(image_info, axis=0)
        color_names = []
        for col in (colors):
            color_names.append('Unknown Color')
        data = list(map(list, zip(colors[:,0], colors[:,1],colors[:,2],color_names)))
        new_df = pd.DataFrame(data,columns=['Red','Green','Blue', "Name"])
        colfig = px.scatter_3d(new_df, x='Red', y='Green', z='Blue',
                    color='Blue', title= TITLE)
        return colfig

In [3]:
immy = Image.open("rainb.jpg")
imageClustered = ClusterImage(immy)

immy.save("saved.png")
# print (imageClustered.image_array)

imageClustered.get_clustered_image(10)

imageClustered.cluster_image_obj.save("savedCluster.png")

100%|██████████| 264600/264600 [00:00<00:00, 338364.28it/s]


In [5]:

image_info = imageClustered.image_array
colors = np.unique(image_info, axis=0)
color_names = []
for col in (colors):
    color_names.append('Unknown Color')
data = list(map(list, zip(colors[:,0], colors[:,1],colors[:,2],color_names)))
new_df = pd.DataFrame(data,columns=['Red','Green','Blue', "Name"])
colfig = px.scatter_3d(new_df, x='Red', y='Green', z='Blue',
            color='Blue', title= "test")
colfig

In [ ]:
colors

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter3d(
    x= colors[:,0],
    y=colors[:,1],
    z=colors[:,2],
    mode='markers',
    marker=dict(
        size=len(colors),
        color=colors, #set color equal to a variable
        # colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()